In [1]:
import os
import cv2
import numpy as np
from scipy.stats import mode
import tensorflow as tf

# 0. Настройка основных параметров

In [2]:
# Main folder path
PROJECT_FOLDER = '.'

# Other folders
DATA_FOLDER = os.path.join(PROJECT_FOLDER, 'dataset')
MODEL_PATH = os.path.join(PROJECT_FOLDER, 'ModelTF.h5')

In [3]:
# Model params
BATCH_SIZE = 100
IMAGE_SIZE = (250, 250)  # im_height, im_width

# 1. Загрузка модели

In [4]:
MyCnn = tf.keras.models.load_model(
    MODEL_PATH, custom_objects=None, compile=True, options=None
)

# 2. Скоринг видео

In [5]:
def predict_video(model, video_path, th=0.011):
    cap = cv2.VideoCapture(video_path)
    batch = []
    probs_0 = []  # accident
    probs_1 = []  # no accident

    while(cap.isOpened()):
        ret, frame = cap.read()
        if len(batch) < BATCH_SIZE and ret == True:
            batch.append(cv2.resize(frame, IMAGE_SIZE))
        elif len(batch) >= BATCH_SIZE or ret == False:
            batch = tf.convert_to_tensor(batch, dtype=None, dtype_hint=None, name=None)
            predictions = model.predict(batch)

            probs_0.append(predictions[:,0])
            probs_1.append(predictions[:,1])

            batch = []

        if ret == False:
            break

    cap.release()

    probs_0, probs_1 = np.concatenate(probs_0), np.concatenate(probs_1)
    evr = probs_0[probs_0 >= mode(probs_0)[0][0]].std()

    if evr >= th:
        print('Авария!')

    else:
        print('Нет аварии!')

In [6]:
VIDEOS_FOLDER = os.path.join(DATA_FOLDER, 'videos')
VIDEO_PATH = os.path.join(VIDEOS_FOLDER, '1.mp4')

predict_video(MyCnn, VIDEO_PATH)

Авария!
